In [6]:
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from chunking import getObjDf

%matplotlib inline

In [7]:
train_meta = pd.read_csv ('/modules/cs342/Assignment2/training_set_metadata.csv',header=0)
train_data = pd.read_csv ('/modules/cs342/Assignment2/training_set.csv',header=0)
test_meta = pd.read_csv('/modules/cs342/Assignment2/test_set_metadata.csv',header=0)

In [22]:
def addFeatures(df,metaDf):
    tmc = metaDf.copy()
    aggregations = {
        'flux': {
            'flux_max':'max',
            'flux_min':'min',
            'flux_mean':'mean',
            'flux_med':'median'
        }
    }
    totalDf = df.groupby('object_id').agg(aggregations)
    totalDf.columns = totalDf.columns.droplevel(level=0)
    tmc = tmc.set_index('object_id').join(totalDf)
    return tmc

In [ ]:
def getNextChunk():
    

In [10]:
def addFeaturesTest(metaDf):
    test_data = pd.read_csv('/modules/cs342/Assignment2/test_set.csv',header=0,chunksize=3000000)
    tmc = metaDf.copy()
    tmc = tmc.set_index('object_id')
    aggregations = {
        'flux': {
            'flux_max':'max',
            'flux_min':'min',
            'flux_mean':'mean',
            'flux_med':'median'
        }
    }
    count=0
    dflist = []
    iterdf = iter(test_data)
    prevdf = next(iterdf)
    breakvar = 0
    while True:
        if breakvar == 1: break
        df = next(iterdf,None)
        if df is None:
            breakvar = 1
#         print "last index is "+str(prevdf.iloc[-1]['object_id'])
#         print "first index is "+str(df.iloc[0]['object_id'])
        if breakvar==0 and prevdf.iloc[-1]['object_id'] == df.iloc[0]['object_id']:
            x = prevdf.iloc[-1]['object_id'] 
            truthT = df['object_id']==x
            prevdf = pd.concat([prevdf,df[truthT]])
            df = df[~truthT]
        count+=1
        totalDf = prevdf.groupby('object_id').agg(aggregations)
        totalDf.columns = totalDf.columns.droplevel(level=0)
        obj = totalDf.index.values
#         print obj
        tempdf = tmc.loc[obj].join(totalDf)
        dflist.append(tempdf)
#         print tempdf
#         print tmc
#         print totalDf
#         if count == 10:
#             break
        if df is not None:
            prevdf = df.copy()     
        if count%15 == 0:print "10% more completed"
    print "joining..."
    tmc = pd.concat(dflist)
    print "complete"
    return tmc
#     tmc = tmc.set_index('object_id').join(totalDf)
#     return tmc

In [11]:
featMeta = addFeaturesTest(test_meta)

10% more completed
10% more completed
10% more completed
10% more completed
10% more completed
10% more completed
10% more completed
10% more completed
10% more completed
10% more completed
joining...
complete


In [15]:
featMeta

,ra,decl,gal_l,gal_b,ddf,hostgal_specz,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,flux_max,flux_min,flux_mean,flux_med
object_id,,,,,,,,,,,,,,
13,34.453125,-5.229529,169.987075,-59.956185,1,0.3048,0.3193,0.0542,41.1123,0.019,42.765503,-12.680235,3.997127,0.616561
14,33.398438,-4.331149,167.226341,-59.936551,1,NaN,0.6323,0.0179,42.8774,0.018,14.839427,-11.142164,0.884047,0.072856
17,348.529419,-61.755440,321.293980,-51.763351,1,NaN,0.8297,0.0605,43.6000,0.016,16.761280,-14.202744,0.791032,0.458391
23,34.804688,-5.829153,171.307861,-60.174401,1,NaN,0.6533,0.1479,42.9640,0.023,28.061138,-12.631923,0.970396,0.465986
34,351.321442,-64.198746,317.458993,-50.429931,1,0.4557,0.4617,0.0122,42.0540,0.023,124.475609,-13.239577,4.580070,0.301366
35,35.332031,-5.979157,172.286722,-59.931743,1,NaN,0.8388,0.0375,43.6290,0.022,34.847000,-9.032743,1.800237,0.589795
43,0.574468,-45.981140,327.041068,-68.778764,1,NaN,0.6669,0.0546,43.0186,0.006,23.771006,-10.711065,0.533671,0.096750
50,0.574468,-45.981140,327.041068,-68.778764,1,NaN,1.4663,0.0529,45.1281,0.006,12.179604,-21.568415,0.012805,0.009715
60,346.562500,-63.448284,320.824720,-49.866957,1,NaN,0.9462,0.0116,43.9519,0.021,28.675814,-21.433050,1.480276,0.087896


In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalise=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Reds):
    
    if normalise:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
#         print("Normalised confusion matrix")
#     else:
#         print('Confusion matrix, without normalization')

#     print(cm)
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalise else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True Class')
    plt.xlabel('Predicted Class')
    plt.tight_layout()
    plt.show()


In [ ]:
features = ['hostgal_photoz', 'flux_med', 'hostgal_photoz_err', 'flux_min', 'flux_max', 'flux_mean']
X = addFeatures(train_data,train_meta)
X_train, X_test = train_test_split(X, random_state=0)
classifier = RandomForestClassifier(n_estimators = 100, criterion='entropy', max_features = 'sqrt', random_state = 0, n_jobs = -1)
classifier.fit(X_train[features].values, X_train['target'].values.ravel())
X_pred = classifier.predict(featMeta[features].values)

In [ ]:
X_pred = classifier.predict(X_test[features].values)
cnf_matrix = confusion_matrix(X_test['target'], X_pred)
np.set_printoptions(precision=2)
class_names = train_meta['target'].unique()
# Plot non-normalized confusion matrix
plt.figure(figsize=(8,7))
plot_confusion_matrix(cnf_matrix, classes=class_names,normalise=True,title='Normalised Confusion Matrix')
plt.show()

In [ ]:

test_data = pd.read_csv('/modules/cs342/Assignment2/test_set.csv',header=0)

NotFittedError: Estimator not fitted, call `fit` before exploiting the model.